<a href="https://colab.research.google.com/github/KatrinaS813/CLWebsite/blob/main/finetuning_twinkle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversation models finetuning

In this notebook we will finetuning an instruction / chat model to behave like Paul Graham. https://www.paulgraham.com/

Adapted from unsloth notebooks, if something is broken check on:
https://unsloth.ai/

### Installation

In [ ]:
%%capture
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

### Load base model

In [20]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

try:
    hf_token = userdata.get('HF_ACCESS_TOKEN')
    if hf_token:
        login(token=hf_token)
    else:
        print("Hugging Face access token not found in Colab secrets.")
except Exception as e:
    print(f"Error during Hugging Face login: {e}")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "twinkle-ai/Llama-3.2-3B-F1-Instruct",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token=userdata.get('HF_ACCESS_TOKEN')
)

Error during Hugging Face login: name 'login' is not defined
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

twinkle-ai/Llama-3.2-3B-F1-Instruct does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [21]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"},
]

formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(formatted_text)


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the capital of France?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




### Add lora to base model and patch with Unsloth

In [22]:
# More info about parameters: https://huggingface.co/docs/peft/v0.11.0/en/package_reference/lora#peft.LoraConfig

target_modules =  ["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"]

# When adding special tokens
train_embeddings = False

if train_embeddings:
  # you run out of memory on colab if you do this
  # target_modules = target_modules + ["lm_head", "embed_tokens"]
  # so if you are on colab and added new tokens instead do
  target_modules = target_modules + ["lm_head"]


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # rank of lora matrices according to paper not much loss when set relatively low
    target_modules = target_modules, # On which modules of the llm the lora weights are used
    lora_alpha = 16,  # scales the weights of the adapters (more influence on base model), 16 was recommended on reddit
    lora_dropout = 0, # Default on 0.05 in tutorial but unsloth says 0 is better
    bias = "none",   # "none" is optimized
    use_gradient_checkpointing = "unsloth", #"unsloth" for very long context, decreases vram
    random_state = 3407,
    use_rslora = False, # scales lora_alpha with 1/sqrt(r), huggingface says this works better
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Create dataset

In [23]:
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]

    # Tokenize texts with padding/truncation (adjust max_length as needed)
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=2048,
        return_tensors=None,  # keep as dict of lists
    )

    # Usually labels = input_ids for causal language modeling
    labels = [ids.copy() for ids in tokenized["input_ids"]]

    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": labels,
    }

import pandas as pd
dataset_json = pd.read_json("hf://datasets/pookie3000/pg_chat/pg_chat_combined.jsonl", lines=True)


from datasets import Dataset
dataset = Dataset.from_pandas(dataset_json, split = 'train')
dataset = dataset.map(formatting_prompts_func, batched = True,)
dataset = dataset.remove_columns("conversations")

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

In [24]:
for i, sample in enumerate(dataset):
    print(f"\n------ Sample {i + 1} ----")
    print(sample["text"])
    if i > 2:
      break



------ Sample 1 ----


KeyError: 'text'

### Train the model


In [25]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True, )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    data_collator = data_collator,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 5,
        #max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        remove_unused_columns=False
    ),
)

In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 484 | Num Epochs = 5 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,5.019300
2,5.750200
3,6.464000
4,5.058000
5,4.963000
6,5.215600
7,5.325100
8,5.039300
9,4.717600
10,4.612300


/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68356c8b-47b2b28815d16f0f7bf4e83b;bd01dbc4-c696-4663-962f-aaee5044ae8d)

Cannot access gated repo for url https://huggingface.co/twinkle-ai/Llama-3.2-3B-F1-Instruct/resolve/main/config.json.
Access to model twinkle-ai/Llama-3.2-3B-F1-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in twinkle-ai/Llama-3.2-3B-F1-Instruct.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in twinkle-ai/Llama-3.2-3B-F1-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


### Inference


In [27]:
from unsloth import FastLanguageModel
from transformers import TextStreamer
import torch

FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "How to do a startup?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

How to do a startup?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"Ah, you want to know how to do a startup, huh? Well, let me tell you, it's not for the faint of heart. It's hard, it's messy, and it's a lot of work. But if you're willing to put in the sweat equity, I'd say go for it. Just don't say I didn't warn you.

First of all, you gotta want to start a startup. If you're just looking for a easy way to make money, then this isn't the life for you. You gotta be willing to put in the long hours, the late nights, and the weekends


### Save lora adapter

This is both useful for inference and if you want to load the model again

In [ ]:
model.push_to_hub(
    "KatrinaSky/llama_paul",
    tokenizer,
    token = userdata.get('HF_ACCESS_TOKEN')
)

/usr/local/lib/python3.11/dist-packages/peft/utils/other.py:1110: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-68356cc2-687f4b1d38fe814b63b5998e;a321c91f-cc2c-471f-adca-78db3395a60e)

Cannot access gated repo for url https://huggingface.co/twinkle-ai/Llama-3.2-3B-F1-Instruct/resolve/main/config.json.
Access to model twinkle-ai/Llama-3.2-3B-F1-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in. - silently ignoring the lookup for the file config.json in twinkle-ai/Llama-3.2-3B-F1-Instruct.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:236: UserWarning: Could not find a config file in twinkle-ai/Llama-3.2-3B-F1-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


### Merge model with lora weights and save to gguf

You can then do inference locally with Ollama or llama.cpp

##### Popular quantization methods

- **q4_k_m**  
  4bit quantization. Low memory. All models you pull with ollama uses this quantization.
- **q8_0**  
  8bit quantization. Medium memory.
- **f16**  
  16 bit quantization. A lot of models are already in 16 bit so then no quantization happens
- **not_quantized**  
  Often same as f16.

In [ ]:
model.push_to_hub_gguf(
    "Meta-Llama-3.1-8B-q4_k_m-paul-graham-guide-GGUF",
    tokenizer,
    quantization_method = "q4_k_m",
    token = userdata.get('HF_ACCESS_TOKEN')
  )

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.95 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 12.58it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [02:06<00:00,  3.95s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF into f16 GGUF format.
The output location will be /content/Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: 

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/pookie3000/Meta-Llama-3.1-8B-Instruct-Paul-Graham-q4_k_m-GGUF


### Load model and saved lora adapters

For if you want to continue finetuning or want to do inference using the model in safetensor format.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "pookie3000/Meta-Llama-3.1-8B-Instruct-Paul-Graham-LORA",
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
    token=userdata.get('HF_ACCESS_TOKEN')
)

FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "What is your job?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128, use_cache = True)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]